<a href="https://colab.research.google.com/github/epicskills1/Final_yr_Project/blob/main/Finalyr_proj_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import io
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

# --- Load data ---
y_tr = pd.read_csv('/content/tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv('/content/tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv('/content/tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
x_te_dense = pd.read_csv('/content/tox21_dense_test.csv.gz', index_col=0, compression="gzip").values
x_tr_sparse = io.mmread('/content/tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread('/content/tox21_sparse_test.mtx.gz').tocsc()

# --- Combine dense + sparse features ---
sparse_col_idx = np.asarray((x_tr_sparse > 0).mean(0) > 0.05).ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].toarray()])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].toarray()])

# --- Normalize ---
scaler = StandardScaler()
x_tr = np.tanh(scaler.fit_transform(x_tr))
x_te = np.tanh(scaler.transform(x_te))

# --- Define NN model ---
def create_model(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim, kernel_regularizer=l2(1e-5)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.4),
        Dense(128, kernel_regularizer=l2(1e-5)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

# --- Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# --- Store metrics ---
auc_nn, auc_xgb, auc_lgb, auc_weighted, auc_stacking = [], [], [], [], []

for target in y_tr.columns:
    print(f"\n🔬 Training on assay: {target}")
    valid_rows = np.isfinite(y_tr[target]).values
    x_target, y_target = x_tr[valid_rows], y_tr[target][valid_rows]

    # SMOTE balancing
    x_balanced, y_balanced = SMOTE(random_state=42).fit_resample(x_target, y_target)
    x_train, x_val, y_train, y_val = train_test_split(x_balanced, y_balanced, test_size=0.2, random_state=42)

    # --- NN ---
    model = create_model(input_dim=x_tr.shape[1])
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['AUC'])
    model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=100, batch_size=32, callbacks=[early_stopping, reduce_lr], verbose=0)

    # --- XGBoost ---
    xgb_model = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.05,
                              subsample=0.8, colsample_bytree=0.8,
                              use_label_encoder=False, eval_metric='logloss', random_state=42)
    xgb_model.fit(x_train, y_train)

    # --- LightGBM ---
    lgb_model = LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)
    lgb_model.fit(x_train, y_train)

    # --- Test set ---
    valid_test_rows = np.isfinite(y_te[target]).values
    x_test_target = x_te[valid_test_rows]
    y_test = y_te[target][valid_test_rows].values

    y_pred_nn = model.predict(x_test_target).ravel()
    y_pred_xgb = xgb_model.predict_proba(x_test_target)[:, 1]
    y_pred_lgb = lgb_model.predict_proba(x_test_target)[:, 1]

    # --- Weighted Ensemble ---
    auc_nn_val = roc_auc_score(y_val, model.predict(x_val).ravel())
    auc_xgb_val = roc_auc_score(y_val, xgb_model.predict_proba(x_val)[:, 1])
    auc_lgb_val = roc_auc_score(y_val, lgb_model.predict_proba(x_val)[:, 1])

    sum_auc = auc_nn_val + auc_xgb_val + auc_lgb_val
    w_nn = auc_nn_val / sum_auc
    w_xgb = auc_xgb_val / sum_auc
    w_lgb = auc_lgb_val / sum_auc

    y_pred_weighted = w_nn * y_pred_nn + w_xgb * y_pred_xgb + w_lgb * y_pred_lgb

    # --- Stacking Ensemble ---
    stack_train = np.vstack([model.predict(x_val).ravel(),
                             xgb_model.predict_proba(x_val)[:, 1],
                             lgb_model.predict_proba(x_val)[:, 1]]).T
    meta_model = LogisticRegression().fit(stack_train, y_val)
    stack_test = np.vstack([y_pred_nn, y_pred_xgb, y_pred_lgb]).T
    y_pred_stack = meta_model.predict_proba(stack_test)[:, 1]

    # --- AUCs ---
    auc_nn.append(roc_auc_score(y_test, y_pred_nn))
    auc_xgb.append(roc_auc_score(y_test, y_pred_xgb))
    auc_lgb.append(roc_auc_score(y_test, y_pred_lgb))
    auc_weighted.append(roc_auc_score(y_test, y_pred_weighted))
    auc_stacking.append(roc_auc_score(y_test, y_pred_stack))

    print(f"{target}:")
    print(f"  ➤ NN AUC         = {auc_nn[-1]:.3f}")
    print(f"  ➤ XGBoost AUC    = {auc_xgb[-1]:.3f}")
    print(f"  ➤ LightGBM AUC   = {auc_lgb[-1]:.3f}")
    print(f"  ➤ Weighted AUC   = {auc_weighted[-1]:.3f}")
    print(f"  ➤ Stacking AUC   = {auc_stacking[-1]:.3f}")

# --- Final Summary ---
print("\n📊 Final Average AUCs:")
print(f"  ➤ Neural Network : {np.mean(auc_nn):.3f}")
print(f"  ➤ XGBoost        : {np.mean(auc_xgb):.3f}")
print(f"  ➤ LightGBM       : {np.mean(auc_lgb):.3f}")
print(f"  ➤ Weighted Ens.  : {np.mean(auc_weighted):.3f}")
print(f"  ➤ Stacking Ens.  : {np.mean(auc_stacking):.3f}")



🔬 Training on assay: NR.AhR


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:54:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 5946, number of negative: 5990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.354879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 336670
[LightGBM] [Info] Number of data points in the train set: 11936, number of used features: 1629
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498157 -> initscore=-0.007373
[LightGBM] [Info] Start training from score -0.007373
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
 1/94 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
SR.p53:
  ➤ NN AUC         = 0.823
  ➤ XGBoost AUC    = 0.829
  ➤ LightGBM AUC   = 0.846
  ➤ Weighted AUC   = 0.856
  ➤ Stacking AUC   = 0.856

📊 Final Average AUCs:
  ➤ Neural Network : 0.802
  ➤ XGBoost        : 0.816
  ➤ LightGBM       : 0.820
  ➤ Weighted Ens.  : 0.830
  ➤ Stacking Ens.  : 0.830


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
pip install -U xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
